In [1]:
import json

with open('/Users/leon/Documents/GitHub/VCTEVA/Test/val:f567361b-8d9d-4f3e-84f6-090ae8499b45.json', 'r') as f:
    data = json.load(f)

In [2]:
data[1]

{'platformGameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
 'configuration': {'teams': [{'teamId': {'value': 21},
    'playersInTeam': [{'value': 6},
     {'value': 7},
     {'value': 8},
     {'value': 9},
     {'value': 10}],
    'name': 'Red'},
   {'teamId': {'value': 22},
    'playersInTeam': [{'value': 1},
     {'value': 2},
     {'value': 3},
     {'value': 4},
     {'value': 5}],
    'name': 'Blue'}],
  'selectedMap': {'id': 'UNKNOWN',
   'fallback': {'guid': '/Game/Maps/Canyon/Canyon', 'displayName': 'Canyon'}},
  'players': [{'accountId': {'type': 'RIOT_ID',
     'value': '8b24051c-f127-54e4-9b6f-ea279eb6a105'},
    'playerId': {'value': 1},
    'selectedAgent': {'fallback': {'guid': '601DBBE7-43CE-BE57-2A40-4ABD24953621',
      'displayName': ''},
     'type': 'UNKNOWN'},
    'type': 'HUMAN',
    'displayName': 'GLD aNNja',
    'tagLine': 'eProd'},
   {'accountId': {'type': 'RIOT_ID',
     'value': '5f2b9283-ec12-5bcf-942d-fabd0f89fbae'},
    'playerId': {'value': 2},
   

In [3]:
def generate_round_summary(data):
    round_started = []
    round_ended = []

    # Extract roundStarted and roundEnded events
    for item in data:
        if 'roundStarted' in item:
            round_started.append({'roundNumber': item['roundStarted']['roundNumber'], 'startTime': item['metadata']['wallTime']})
        elif 'roundEnded' in item:
            round_ended.append({'roundNumber': item['roundEnded']['roundNumber'], 'endTime': item['metadata']['wallTime']})

    # Match roundStarted and roundEnded events
    rounds = []
    for i in range(len(round_ended)):
        rounds.append({
            'roundNumber': round_started[i]['roundNumber'],
            'startTime': round_started[i]['startTime'],
            'endTime': round_ended[i]['endTime']
        })

    # Handle the case where there is an extra roundStarted without a matching roundEnded
    if len(round_started) > len(round_ended):
        rounds.append({
            'roundNumber': round_started[-1]['roundNumber'],
            'startTime': round_started[-1]['startTime'],
            'endTime': 'Game End'
        })

    return rounds

# Generate the round summary
round_summary = generate_round_summary(data)

# Print the round summary
for round_info in round_summary:
    print(f"Round: {round_info['roundNumber']}, Start Time: {round_info['startTime']}, End Time: {round_info['endTime']}")

Round: 1, Start Time: 2022-11-18T15:32:41.041Z, End Time: 2022-11-18T15:45:17.813Z
Round: 2, Start Time: 2022-11-18T15:45:18.371Z, End Time: 2022-11-18T15:47:08.375Z
Round: 3, Start Time: 2022-11-18T15:47:08.693Z, End Time: 2022-11-18T15:48:19.246Z
Round: 4, Start Time: 2022-11-18T15:48:19.612Z, End Time: 2022-11-18T15:50:19.074Z
Round: 5, Start Time: 2022-11-18T15:50:19.469Z, End Time: 2022-11-18T15:52:38.364Z
Round: 6, Start Time: 2022-11-18T15:52:38.693Z, End Time: 2022-11-18T15:54:38.206Z
Round: 7, Start Time: 2022-11-18T15:54:38.608Z, End Time: 2022-11-18T15:56:17.050Z
Round: 8, Start Time: 2022-11-18T15:56:17.475Z, End Time: 2022-11-18T15:57:52.214Z
Round: 9, Start Time: 2022-11-18T15:57:52.618Z, End Time: 2022-11-18T16:01:18.788Z
Round: 10, Start Time: 2022-11-18T16:01:19.105Z, End Time: 2022-11-18T16:03:18.465Z
Round: 11, Start Time: 2022-11-18T16:03:18.819Z, End Time: 2022-11-18T16:04:34.818Z
Round: 12, Start Time: 2022-11-18T16:04:35.170Z, End Time: 2022-11-18T16:06:47.016Z
R

TODO last round valid

In [4]:
def get_game_decided(data):
    for item in data:
        if 'gameDecided' in item:
            return item
    return ("Incomplete data")

def generate_round_details(data):
    game_decided = get_game_decided(data)
    if not game_decided:
        return "No gameDecided event found"

    spike_mode = game_decided['gameDecided']['spikeMode']
    completed_rounds = spike_mode['completedRounds']

    rounds_summary = generate_round_summary(data)

    # Combine round summary with additional round details
    detailed_rounds = []
    for round_info in rounds_summary:
        round_number = round_info['roundNumber']
        # Find corresponding round in completedRounds
        matching_round = next((r for r in completed_rounds if r['roundNumber'] == round_number), None)

        if matching_round:
            round_info.update({
                'attackingTeam': matching_round['spikeModeResult']['attackingTeam']['value'],
                'defendingTeam': matching_round['spikeModeResult']['defendingTeam']['value'],
                'winningTeam': matching_round['winningTeam']['value'],
                # 'cause': matching_round['spikeModeResult']['cause']
            })

        detailed_rounds.append(round_info)

    # Set winning team for the last round if necessary
    if len(rounds_summary) > len(completed_rounds):
        last_round = detailed_rounds[-1]
        last_round.update({
            'attackingTeam': spike_mode['attackingTeam']['value'],
            'defendingTeam': spike_mode['defendingTeam']['value'],
            'winningTeam': spike_mode['winningTeam']['value'],
            # 'cause': 'Game End'
        })

    return detailed_rounds


In [5]:
generate_round_details(data)

[{'roundNumber': 1,
  'startTime': '2022-11-18T15:32:41.041Z',
  'endTime': '2022-11-18T15:45:17.813Z',
  'attackingTeam': 21,
  'defendingTeam': 22,
  'winningTeam': 21},
 {'roundNumber': 2,
  'startTime': '2022-11-18T15:45:18.371Z',
  'endTime': '2022-11-18T15:47:08.375Z',
  'attackingTeam': 21,
  'defendingTeam': 22,
  'winningTeam': 21},
 {'roundNumber': 3,
  'startTime': '2022-11-18T15:47:08.693Z',
  'endTime': '2022-11-18T15:48:19.246Z',
  'attackingTeam': 21,
  'defendingTeam': 22,
  'winningTeam': 22},
 {'roundNumber': 4,
  'startTime': '2022-11-18T15:48:19.612Z',
  'endTime': '2022-11-18T15:50:19.074Z',
  'attackingTeam': 21,
  'defendingTeam': 22,
  'winningTeam': 21},
 {'roundNumber': 5,
  'startTime': '2022-11-18T15:50:19.469Z',
  'endTime': '2022-11-18T15:52:38.364Z',
  'attackingTeam': 21,
  'defendingTeam': 22,
  'winningTeam': 21},
 {'roundNumber': 6,
  'startTime': '2022-11-18T15:52:38.693Z',
  'endTime': '2022-11-18T15:54:38.206Z',
  'attackingTeam': 21,
  'defendingT

In [6]:
def extract_configuration_details(data):
    first_config = None
    for event in data:
        if 'configuration' in event:
            first_config = event
            break

    if not first_config:
        return "No configuration event found"

    platformGameId = event["platformGameId"]
    # Extract selected map
    selected_map = first_config["configuration"]["selectedMap"]["fallback"]["displayName"]

    # Extract team ID and players in team
    teams = first_config["configuration"]["teams"]
    team_info = []
    for team in teams:
        team_id = team["teamId"]["value"]
        players_in_team = [player["value"] for player in team["playersInTeam"]]
        team_info.append({"teamId": team_id, "playersInTeam": players_in_team})

    # Extract current round number and attacking team value
    current_round = first_config["configuration"]["spikeMode"]["currentRound"]
    attacking_team_value = first_config["configuration"]["spikeMode"]["attackingTeam"]["value"]

    return {
        "platformGameId":platformGameId,
        "selectedMap": selected_map,
        "teams": team_info,
        "currentRound": current_round,
        "attackingTeam": attacking_team_value
    }

In [7]:
# Helper function to map player ID using the platformGameId and participantMapping
def map_player_id(player_id, mapping):
    # Find the mapping for the platformGameId
    return mapping['participantMapping'].get(str(player_id), player_id)  # 

def map_team_id(team_id, mapping):
    # Find the mapping for the platformGameId
    return mapping['teamMapping'].get(str(team_id))  # 



mapping_file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/mapping_data.json"

with open(mapping_file_path, 'r') as mf:
        mapping_data = json.load(mf)

for mapping in mapping_data:
    if mapping['platformGameId'] == data[1]['platformGameId']:
        mapping_data = mapping

In [8]:
def generate_game_summary(data,mapping_data):
    config_details = extract_configuration_details(data)
    if isinstance(config_details, str):  # Handle error message
        return config_details

    round_details = generate_round_details(data)
    if isinstance(round_details, str):  # Handle error message
        return round_details

    # Combine all details into a single JSON
    game_summary = {
        "platformGameId":config_details["platformGameId"],
        "selectedMap": config_details["selectedMap"],
        "teams": config_details["teams"],
        "round_details": round_details
    }
    for team in game_summary["teams"]:
        team["teamId"]= map_team_id(team["teamId"], mapping_data)
        for i in range(len(team["playersInTeam"])):
            team["playersInTeam"][i] = map_player_id(team["playersInTeam"][i], mapping_data)
    for round in game_summary["round_details"]:
        round['attackingTeam']=map_team_id(round["attackingTeam"], mapping_data)
        round['defendingTeam']=map_team_id(round["defendingTeam"], mapping_data)
        round['winningTeam']=map_team_id(round["winningTeam"], mapping_data)
    return game_summary

In [9]:
game_summary = generate_game_summary(data,mapping_data)
game_summary

{'platformGameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
 'selectedMap': 'Canyon',
 'teams': [{'teamId': '106976745966764918',
   'playersInTeam': ['106977394394708775',
    '106977392050144626',
    '106977393332501282',
    '106977390661334163',
    '106977457782256833']},
  {'teamId': '107021308990636196',
   'playersInTeam': ['107025879641342377',
    '107186009230387255',
    '107186009028237077',
    '107186009533585182',
    '109080898754120338']}],
 'round_details': [{'roundNumber': 1,
   'startTime': '2022-11-18T15:32:41.041Z',
   'endTime': '2022-11-18T15:45:17.813Z',
   'attackingTeam': '106976745966764918',
   'defendingTeam': '107021308990636196',
   'winningTeam': '106976745966764918'},
  {'roundNumber': 2,
   'startTime': '2022-11-18T15:45:18.371Z',
   'endTime': '2022-11-18T15:47:08.375Z',
   'attackingTeam': '106976745966764918',
   'defendingTeam': '107021308990636196',
   'winningTeam': '106976745966764918'},
  {'roundNumber': 3,
   'startTime': '2022-11-18T15:

In [10]:
def get_round_number_from_event(rounds_summary, event):
    wall_time = event['metadata']['wallTime']

    for round_info in rounds_summary:
        if round_info['endTime'] == 'Game End':
            if round_info['startTime'] <= wall_time:
                return round_info['roundNumber']
        elif round_info['startTime'] <= wall_time <= round_info['endTime']:
            return round_info['roundNumber']
    return None


In [11]:
import json
import pandas as pd

# Load the JSON data for the game events and the mapping file
file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/val:f567361b-8d9d-4f3e-84f6-090ae8499b45.json"
mapping_file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/mapping_data.json"

with open(file_path, 'r') as f:
    data = json.load(f)

with open(mapping_file_path, 'r') as mf:
    mapping_data = json.load(mf)
    
# Helper function to map player ID using the platformGameId and participantMapping
def map_player_id(platform_game_id, player_id, mappings):
    # Find the mapping for the platformGameId
    for mapping in mappings:
        if mapping['platformGameId'] == platform_game_id:
            # Return the corresponding esports player ID from participantMapping
            return mapping['participantMapping'].get(str(player_id), player_id)  # Return player_id if not found
    return player_id  # Return the original player_id if platformGameId is not found



In [12]:
def generate_kda_per_player(game_summary, data, mapping_data):
        # Ensure rounds_summary is a list of dictionaries
    rounds_summary = game_summary['round_details']
    platform_game_id = game_summary['platformGameId']
    selected_map = game_summary['selectedMap']

    # Initialize a list to store KDA information for each player
    kda_per_player_list = []

    # Iterate through events to collect KDA information
    for event in data:
        if 'playerDied' in event:
            round_number = get_round_number_from_event(rounds_summary, event)

            if round_number is None:
                continue

            player_died_data = event['playerDied']
            deceased_id = player_died_data.get('deceasedId', {}).get('value')
            killer_id = player_died_data.get('killerId', {}).get('value')

            # Map deceased and killer IDs using the participantMapping
            mapped_deceased_id = map_player_id(platform_game_id,deceased_id, mapping_data) if deceased_id else None
            mapped_killer_id = map_player_id(platform_game_id,killer_id, mapping_data) if killer_id else None

            # Initialize player KDA if not already done
            def initialize_player_kda(player_id):
                existing_player = next((player for player in kda_per_player_list if player['PlayerId'] == player_id), None)
                if existing_player is None:
                    kda_per_player_list.append({
                        "PlayerId": player_id,
                        "GameId": platform_game_id,
                        "Map": selected_map,
                        "RoundInfo": []
                    })
                return next(player for player in kda_per_player_list if player['PlayerId'] == player_id)

            if mapped_deceased_id:
                deceased_player = initialize_player_kda(mapped_deceased_id)
                deceased_player['RoundInfo'].append({'RoundNumber': round_number, 'Deaths': 1, 'Kills': 0, 'Assists': 0})

            if mapped_killer_id:
                killer_player = initialize_player_kda(mapped_killer_id)
                killer_player['RoundInfo'].append({'RoundNumber': round_number, 'Deaths': 0, 'Kills': 1, 'Assists': 0})

            # Check for assistants and update their assist counts
            if 'assistants' in event['playerDied']:
                for assistant in event['playerDied']['assistants']:
                    assistant_id = assistant['assistantId']['value']
                    mapped_assistant_id = map_player_id( platform_game_id,assistant_id, mapping_data)
                    assistant_player = initialize_player_kda(mapped_assistant_id)
                    assistant_player['RoundInfo'].append({'RoundNumber': round_number, 'Deaths': 0, 'Kills': 0, 'Assists': 1})

    return kda_per_player_list


In [13]:
generate_kda_per_player(game_summary, data, mapping_data)

[{'PlayerId': '107186009230387255',
  'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
  'Map': 'Canyon',
  'RoundInfo': [{'RoundNumber': 1, 'Deaths': 1, 'Kills': 0, 'Assists': 0},
   {'RoundNumber': 2, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'RoundNumber': 2, 'Deaths': 1, 'Kills': 0, 'Assists': 0},
   {'RoundNumber': 3, 'Deaths': 0, 'Kills': 0, 'Assists': 1},
   {'RoundNumber': 3, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'RoundNumber': 3, 'Deaths': 1, 'Kills': 0, 'Assists': 0},
   {'RoundNumber': 4, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'RoundNumber': 4, 'Deaths': 1, 'Kills': 0, 'Assists': 0},
   {'RoundNumber': 5, 'Deaths': 1, 'Kills': 0, 'Assists': 0},
   {'RoundNumber': 6, 'Deaths': 0, 'Kills': 0, 'Assists': 1},
   {'RoundNumber': 6, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'RoundNumber': 6, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'RoundNumber': 7, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'RoundNumber': 7, 'Deaths': 0, 'Kills': 1, 'Assists': 0},
   {'Rou

In [14]:
def merge_kda_per_round(kda_per_player_list):
    for player in kda_per_player_list:
        filtered_round_info = [round_info for round_info in player['RoundInfo']]
        merged_rounds = {}
        for round_info in filtered_round_info:
            round_number = round_info['RoundNumber']
            if round_number not in merged_rounds:
                merged_rounds[round_number] = {'RoundNumber': round_number, 'Kills': 0, 'Deaths': 0, 'Assists': 0}
            merged_rounds[round_number]['Kills'] += round_info.get('Kills', 0)
            merged_rounds[round_number]['Deaths'] += round_info.get('Deaths', 0)
            merged_rounds[round_number]['Assists'] += round_info.get('Assists', 0)

        # Add a summary KDA for each player
        total_kills = sum(info['Kills'] for info in merged_rounds.values())
        total_deaths = sum(info['Deaths'] for info in merged_rounds.values())
        total_assists = sum(info['Assists'] for info in merged_rounds.values())
        player['RoundInfo'] = [info for info in merged_rounds.values() 
                               if not (info['Kills'] == 0 and info['Deaths'] == 0 and info['Assists'] == 0)]
        player['Summary'] = {'Kills': total_kills, 
                             'Deaths': total_deaths, 
                             'Assists': total_assists}
    return kda_per_player_list

In [15]:
kda_per_player_list = merge_kda_per_round(generate_kda_per_player(game_summary, data, mapping_data))
kda_per_player_list

[{'PlayerId': '107186009230387255',
  'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
  'Map': 'Canyon',
  'RoundInfo': [{'RoundNumber': 1, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 2, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 3, 'Kills': 1, 'Deaths': 1, 'Assists': 1},
   {'RoundNumber': 4, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 5, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 6, 'Kills': 2, 'Deaths': 0, 'Assists': 1},
   {'RoundNumber': 7, 'Kills': 3, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 8, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 9, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 10, 'Kills': 3, 'Deaths': 0, 'Assists': 0},
   {'RoundNumber': 11, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 12, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 13, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 14, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   

In [16]:
def find_player_by_id_and_team(player_data, id_value, home_team_id_value):
    for player in player_data:
        if player["id"] == id_value and player["home_team_id"] == home_team_id_value:
            return player
    return None

In [17]:
def find_team_by_player(game_summary, player_id):
    for team in game_summary['teams']:
        if player_id in team['playersInTeam']:
            return team['teamId']
    return None 

In [18]:
kda_per_player_list

[{'PlayerId': '107186009230387255',
  'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
  'Map': 'Canyon',
  'RoundInfo': [{'RoundNumber': 1, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 2, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 3, 'Kills': 1, 'Deaths': 1, 'Assists': 1},
   {'RoundNumber': 4, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 5, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 6, 'Kills': 2, 'Deaths': 0, 'Assists': 1},
   {'RoundNumber': 7, 'Kills': 3, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 8, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 9, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 10, 'Kills': 3, 'Deaths': 0, 'Assists': 0},
   {'RoundNumber': 11, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 12, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 13, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 14, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   

In [19]:
def merge_player_data(kda_per_player_list, player_file_path, game_summary):
    # Load player data
    with open(player_file_path, 'r') as pf:
        player_data = json.load(pf)
    
    
    for player_kda in kda_per_player_list:
        player_kda["team_id"] = find_team_by_player(game_summary, player_kda["PlayerId"])

        player_info = find_player_by_id_and_team(player_data, player_kda["PlayerId"], player_kda["team_id"])

        player_kda["handle"] =player_info["handle"] 
        player_kda["first_name"] =player_info["first_name"]
        player_kda["last_name"] =player_info["last_name"]
    return kda_per_player_list

In [20]:
import json
player_file_path = "/Users/leon/Documents/GitHub/VCTEVA/Test/players.json"
merge_player_data(kda_per_player_list, player_file_path, game_summary)

[{'PlayerId': '107186009230387255',
  'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
  'Map': 'Canyon',
  'RoundInfo': [{'RoundNumber': 1, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 2, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 3, 'Kills': 1, 'Deaths': 1, 'Assists': 1},
   {'RoundNumber': 4, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 5, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 6, 'Kills': 2, 'Deaths': 0, 'Assists': 1},
   {'RoundNumber': 7, 'Kills': 3, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 8, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 9, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 10, 'Kills': 3, 'Deaths': 0, 'Assists': 0},
   {'RoundNumber': 11, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 12, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 13, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 14, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   

In [23]:
def merge_team_league_data(kda_per_player_list, team_file_path, league_file_path):
    # Load the team data
    with open(team_file_path, 'r') as team_file:
        teams = json.load(team_file)

    # Load the league data
    with open(league_file_path, 'r') as league_file:
        leagues = json.load(league_file)

    # Create a dictionary to map league_id to league details
    league_dict = {league['league_id']: league for league in leagues}

    # List to hold the new matched data
    matched_data = []

    # Match teams with their corresponding leagues
    for team in teams:
        home_league_id = team['home_league_id']
        if home_league_id in league_dict:
            league = league_dict[home_league_id]
            matched_data.append({
                "team_id": team['id'],
                "acronym": team['acronym'],
                "league_id": league['league_id'],
                "team_name": team['name'],
                "league_region": league['region'],
                "league_name": league['name']
            })

    # Remove duplicates and keep only the first match for each team_id
    matched_data_df = pd.DataFrame(matched_data).drop_duplicates(subset='team_id', keep='first')

    # Merge team and league data with KDA data
    kda_df = pd.DataFrame(kda_per_player_list)
    final_df = kda_df.merge(matched_data_df, left_on='team_id', right_on='team_id', how='left')
    
    # Convert the merged DataFrame back to a list of dictionaries
    kda_per_player_list = final_df.to_dict(orient='records')

    return kda_per_player_list


In [24]:
team_file_path='/Users/leon/Documents/GitHub/VCTEVA/Test/teams.json'
league_file_path= '/Users/leon/Documents/GitHub/VCTEVA/Test/leagues.json'
player_league_data = merge_team_league_data(kda_per_player_list, team_file_path, league_file_path)
player_league_data

[{'PlayerId': '107186009230387255',
  'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
  'Map': 'Canyon',
  'RoundInfo': [{'RoundNumber': 1, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 2, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 3, 'Kills': 1, 'Deaths': 1, 'Assists': 1},
   {'RoundNumber': 4, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 5, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 6, 'Kills': 2, 'Deaths': 0, 'Assists': 1},
   {'RoundNumber': 7, 'Kills': 3, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 8, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 9, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 10, 'Kills': 3, 'Deaths': 0, 'Assists': 0},
   {'RoundNumber': 11, 'Kills': 1, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 12, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 13, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   {'RoundNumber': 14, 'Kills': 0, 'Deaths': 1, 'Assists': 0},
   

In [31]:
def add_round_roles_and_results(data,mapping_data,player_file_path, team_file_path, league_file_path,game_summary): 

    player_league_data = merge_team_league_data(kda_per_player_list, team_file_path, league_file_path)

    round_details = game_summary['round_details']
    # Extract team mapping from mapping_data


    # Add round details to the KDA data
    for player in player_league_data:
        player_round_info = player['RoundInfo']
        existing_round_numbers = {round_info['RoundNumber'] for round_info in player_round_info}

        # Find the player's team ID based on the mapping


        # Iterate through all round details and update or add information, excluding the last round
        for round_detail in round_details:
            round_number = round_detail['roundNumber']

            role = "attacking" if player["team_id"] == round_detail['attackingTeam'] else "defending"
            result = "win" if player["team_id"] == round_detail['winningTeam'] else "lose"

            # Update existing rounds
            for round_info in player_round_info:
                if round_info['RoundNumber'] == round_number:
                    round_info['role'] = role
                    round_info['result'] = result
                    break
                else:
                    # Add missing rounds with KDA as 0 if not present
                    if round_number not in existing_round_numbers:
                        player_round_info.append({
                            'RoundNumber': round_number,
                            'Kills': 0,
                            'Deaths': 0,
                            'Assists': 0,
                            'role': role,
                            'result': result
                        })

        # Add game result to summary
        player_round_info.sort(key=lambda x: x['RoundNumber'])
        last_round = round_details[-1]
        final_result = "win" if player["team_id"] == last_round['winningTeam'] else "lose"
        player['Summary']['result'] = final_result

    return player_league_data

In [32]:
add_round_roles_and_results(data,mapping_data,player_file_path, team_file_path, league_file_path,game_summary)

[{'PlayerId': '107186009230387255',
  'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45',
  'Map': 'Canyon',
  'RoundInfo': [{'RoundNumber': 1,
    'Kills': 0,
    'Deaths': 1,
    'Assists': 0,
    'role': 'defending',
    'result': 'lose'},
   {'RoundNumber': 2,
    'Kills': 1,
    'Deaths': 1,
    'Assists': 0,
    'role': 'defending',
    'result': 'lose'},
   {'RoundNumber': 3,
    'Kills': 1,
    'Deaths': 1,
    'Assists': 1,
    'role': 'defending',
    'result': 'win'},
   {'RoundNumber': 4,
    'Kills': 1,
    'Deaths': 1,
    'Assists': 0,
    'role': 'defending',
    'result': 'lose'},
   {'RoundNumber': 5,
    'Kills': 0,
    'Deaths': 1,
    'Assists': 0,
    'role': 'defending',
    'result': 'lose'},
   {'RoundNumber': 6,
    'Kills': 2,
    'Deaths': 0,
    'Assists': 1,
    'role': 'defending',
    'result': 'win'},
   {'RoundNumber': 7,
    'Kills': 3,
    'Deaths': 1,
    'Assists': 0,
    'role': 'defending',
    'result': 'lose'},
   {'RoundNumber': 8,
    'Kill

In [21]:
for player in player_league_data:
    print(player)

{'PlayerId': '107186009230387255', 'GameId': 'val:f567361b-8d9d-4f3e-84f6-090ae8499b45', 'Map': 'Canyon', 'RoundInfo': [{'RoundNumber': 1, 'Kills': 0, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 2, 'Kills': 1, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 3, 'Kills': 1, 'Deaths': 1, 'Assists': 1, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 4, 'Kills': 1, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 5, 'Kills': 0, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 6, 'Kills': 2, 'Deaths': 0, 'Assists': 1, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 7, 'Kills': 3, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 8, 'Kills': 0, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}, {'RoundNumber': 9, 'Kills': 0, 'Deaths': 1, 'Assists': 0, 'role': 'defending', 'result': 'lose'}